In [1]:
from langchain_groq import ChatGroq
import os

In [2]:
import chromadb

In [3]:
GROG_API_KEY = os.environ.get("MY_GROG_API_KEY")

In [4]:
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.5,
    api_key=GROG_API_KEY,
    max_tokens=None,
    max_retries=2
)

In [26]:
#client = chromadb.Client()
#collection = client.create_collection(name="cold_mail")
#collection = client.create_collection(name="cold_email")

Capturing Website Data

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://jobs.nike.com/job/R-43863")
page_data = loader.load().pop().page_content
print(page_data)

In [19]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """ 
    ### SRAPPED TEST FROM WEBSITE
    {page_data}
    ### INSTRUCTION:
    The scrapped text is from the career's page of a website.
    Your job is extract the job postings and return them in JSON format containing following keys: 'role','experience','skills' and 'description'
    Only return valid JSON.
    ### VALID JSON (NO PREAMBLE)
    """
)

In [ ]:
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
jason_parser = JsonOutputParser()
jason_res = jason_parser.parse(res.content)
jason_res

Importing Dataset and Inserting into ChromaDB

In [ ]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df

In [28]:
import uuid
client =chromadb.PersistentClient('portfoliostore')
collection = client.get_or_create_collection('portfolio')

if not collection.count():
    for _,row in df.iterrows():
        collection.add(
            documents=row['Techstack'],
            metadatas={'links' : row['Links']},
            ids=[str(uuid.uuid4())]
        ) 


In [ ]:
collection.query(query_texts=jason_res['skills'],n_results=2).get('metadatas')